In [6]:
import copy
import itertools as it
import numpy as np
import yaml
import os

def expand_config(dict_config):
    keys, values = zip(*dict_config.items())
    permutations_dicts = [dict(zip(keys, v)) for v in it.product(*values)]
    return permutations_dicts

In [7]:
def generate_train_str_config(config):
    config_list = []
    config_list.append('ignore_exceptions=False use_density_based_ue=True')
    
    config_list.append('batch_size={}'.format(config['batch_size']))
    config_list.append('subsample_train_dataset={}'.format(config['subsample_train_dataset']))
    config_list.append('subsample_background_train_dataset={}'.format(config['subsample_background_train_dataset']))
    config_list.append('subsample_eval_dataset={}'.format(config['subsample_eval_dataset']))
    config_list.append('model={}'.format(config['model']))
    if ("mistral" in config['model'].lower()) :
        config_list.append('+allow_newlines=False')
    
    return config_list

In [8]:
def generate_bash(configs, cuda_devices, tasks, generate_func, filename=''):
    full_config = 'cd ../../lm-polygraph'
    j = 0
    n_devices = len(cuda_devices)
    for i, mc_configs in enumerate(configs):
        for conf in expand_config(mc_configs):
            for task_name in tasks:

                base_arg = f'CUDA_VISIBLE_DEVICES={cuda_devices[j%n_devices]} HYDRA_CONFIG=../examples/configs/polygraph_eval_{task_name}.yaml python ./scripts/polygraph_eval device=cuda:0'

                new_task = copy.deepcopy(base_arg)
                args = ' '.join(generate_func[i](conf))
                new_task += f' {args}'
                if (j+1)%n_devices!=0: 
                    new_task += ' &'
                else:
                    new_task += '\nwait'
                full_config += '\n' + new_task if len(full_config) else new_task
                j+=1
                
    with open (f'./scripts/{filename}', 'w') as rsh:
        rsh.write(full_config)

In [9]:
tasks = ['coqa', 'sciq', 'triviaqa', 'mmlu']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [5000],
    'subsample_background_train_dataset': [2000],
    'subsample_eval_dataset': [2000],
    'model': ['lmsys/vicuna-7b-v1.5', 'alpindale/Mistral-7B-v0.2-hf', 'stabilityai/stablelm-2-1_6b', 'meta-llama/Llama-2-7b-chat-hf', 'stabilityai/stablelm-2-12b', 'lmsys/vicuna-13b-v1.5'],
}
    
cuda_devices = [0,1,2,3,4,5,6,7]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p1.sh')

In [10]:
tasks = ['gsm8k', 'xsum', 'wmt14_deen']

train_configs = {
    'batch_size': [1],
    'subsample_train_dataset': [2000],
    'subsample_background_train_dataset': [1000],
    'subsample_eval_dataset': [1000],
    'model': ['lmsys/vicuna-7b-v1.5', 'alpindale/Mistral-7B-v0.2-hf', 'stabilityai/stablelm-2-1_6b', 'stabilityai/stablelm-2-12b', 'lmsys/vicuna-13b-v1.5'],
}
    
cuda_devices = [0,1,2,3,4,5,6,7]

generate_bash([train_configs], cuda_devices, tasks, [generate_train_str_config], 'run_tmd_exps_p2.sh')